In [0]:
DATA_DIR = '/datascope/subaru/data/catalogs/xshooter/'
LIST_FILE = DATA_DIR + '_list.csv'

PROJECT_PATH = '/home/dobos/project/ga_isochrones/python:' + \
    '/home/dobos/project/ga_pfsspec_all/python:' + \
    '/home/dobos/project/pysynphot'

GRID_PATH = '/datascope/subaru/data/pfsspec/models/stellar/grid/phoenix/phoenix_HiRes'

PSF_PATH = '/datascope/subaru/data/pfsspec/subaru/pfs/psf/import/{}.2'
DETECTOR_PATH = '/datascope/subaru/data/pfsspec/subaru/pfs/arms/{}.json'
DETECTORMAP_PATH = '/datascope/subaru/data/pfsspec/drp_pfs_data/detectorMap/detectorMap-sim-{}1.fits'

ARMS = ['b', 'r', 'n', 'mr']

In [0]:
import os, sys, re, glob
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import notebook as tqdm
from astropy.io import fits
from glob import glob

In [0]:
# Allow load project as module
for p in reversed(PROJECT_PATH.split(':')):
    sys.path.insert(0, p)

os.environ['PYTHONPATH'] = PROJECT_PATH.split(':')[0]

In [0]:
%matplotlib inline

In [0]:
%load_ext autoreload

In [0]:
%autoreload 2

# Load model grid

In [0]:
from pfs.ga.pfsspec.core.grid import ArrayGrid
from pfs.ga.pfsspec.stellar.grid import ModelGrid
from pfs.ga.pfsspec.stellar.grid.bosz import Bosz
from pfs.ga.pfsspec.stellar.grid.phoenix import Phoenix

fn = os.path.join(GRID_PATH, 'spectra.h5')
grid = ModelGrid(Phoenix(), ArrayGrid)
grid.preload_arrays = False
grid.load(fn, format='h5')

# Load data

In [0]:
from pfs.ga.pfsspec.survey.xsl import XslSpectrum, XslSurvey

In [0]:
xshooter = XslSurvey()
xshooter.load(os.path.join(DATA_DIR, 'xsl.dat'))

In [0]:
# Spectra that are dereddened / slit corrected
# for i in range(len(xshooter.spectra)):
    # if hasattr(xshooter.spectra[i], 'flux_dered') and xshooter.spectra[i].flux_dered is not None:
    #     print(i)
    # if hasattr(xshooter.spectra[i], 'flux_sc') and xshooter.spectra[i].flux_sc is not None:
    #     print(i)

In [0]:
spec = xshooter.spectra[12]

f, ax = plt.subplots(1, 1, figsize=(3.4, 2.5), dpi=240)

#mask = (790 <= wave) & (wave <= 796)
mask = np.full_like(spec.wave, True, dtype=np.bool)

ax.plot(spec.wave, spec.flux, '-k', lw=0.5)
ax.plot(spec.wave, spec.flux_err, '-r', lw=0.5)
if spec.flux_dered is not None:
    ax.plot(spec.wave, spec.flux_dered, '-b', lw=0.5)
if hasattr(spec, 'flux_sc') and spec.flux_sc is not None:
    ax.plot(spec.wave, spec.flux_sc, '-g', lw=0.5)

ax.set_ylim(0, 1.5 * np.nanquantile(spec.flux[mask], 0.99))

In [0]:
from matplotlib.ticker import ScalarFormatter

f, ax = plt.subplots(1, 1, figsize=(3.4, 3.4), dpi=240)

l = ax.scatter(xshooter.params['T_eff'], xshooter.params['log_g'], c=xshooter.params['Fe_H'], s=1, cmap='jet')
ax.invert_xaxis()
ax.invert_yaxis()
ax.set_xscale('log')
ax.set_xlim(11000, None)
#ax.xaxis.set_major_formatter(ScalarFormatter())
#ax.xaxis.set_minor_formatter(ScalarFormatter())
ax.set_xlabel(r'$T_\mathrm{eff}$')
ax.set_ylabel(r'$\log\,g$')

f.colorbar(l)

# Low metallicity RGB stars

In [0]:
mask = (xshooter.params['Fe_H'] < -1.5) & (xshooter.params['log_g'] < 3.5) & (xshooter.params['T_eff'] < 6500)
mask.sum()

In [0]:
np.where(mask)

# Instrumental resolution

Vernet et al. (2011), Table 4

* https://www.eso.org/sci/facilities/paranal/instruments/xshooter/doc/xshooter_Vernet2011.pdf

XSL observations

"The narrow-slit
widths for the UVB, VIS, and NIR arms of X-shooter were 0".5, 0".7, and 0".6, resulting in resolving powers of R = 9 793, 11 573 and 7 956, respectively. We took wide-slit spectra of the targets
immediately after the narrow-slit exposure, using a 5" wide slit, to flux calibrate the spectra"

Approximate coverage

* UVB: 320 - 540 nm, R = 9793
* VIS: 550 - 1000 nm, R = 11573
* NIR: 1000 - 2500 nm, R = 7956

The PFS arms, as comparison

* B: 380 - 650 nm, R =
* R: 630 - 970 nm, R =
* N: 940 - 1260 nm, R =
* MR: 710 - 8850 nm, R =

In [0]:
# X-Shooter sigma

def xsl_sigma(wave):
    fwhm = np.full_like(wave, np.nan)

    masks = [
        (3200 <= wave) & (wave < 5500),
        (5500 <= wave) & (wave < 10000),
        (10000 <= wave) & (wave < 25000)
    ]
    res = [ 9793, 11573, 7956 ]

    for mask, r in zip(masks, res):
        fwhm[mask] = wave[mask] / r

    return fwhm / (2 * np.sqrt(2 * np.log(2)))

In [0]:
# Load PFS kernel model (ETC)

from pfs.ga.pfsspec.core.obsmod.psf import GaussPsf, PcaPsf

gauss_psf = {}
for arm in ARMS:
    psf = GaussPsf()
    psf.load(os.path.join(PSF_PATH.format(arm), 'gauss.h5'), format='h5')
    gauss_psf[arm] = psf


In [0]:
f, ax = plt.subplots(1, 1, figsize=(3.4, 2.5), dpi=240)

for arm in ARMS:
    ax.plot(gauss_psf[arm].wave, gauss_psf[arm].sigma, lw=0.5)
    ax.plot(gauss_psf[arm].wave, xsl_sigma(gauss_psf[arm].wave), '--k', lw=0.5)

    ax.plot(gauss_psf[arm].wave, np.sqrt(gauss_psf[arm].sigma**2 - xsl_sigma(gauss_psf[arm].wave)**2), '-.k', lw=0.5)

ax.grid()

ax.set_xlabel('wavelength [A]')
ax.set_ylabel('LSF sigma')

# Convolve and resample

In [0]:
spec = xshooter.spectra[12]

mask = (spec.flux_err > 0)
snr = np.median(spec.flux[mask] / spec.flux_err[mask])

spec.snr, snr

In [0]:
deg_psf = {}
for arm in ARMS:
    psf = GaussPsf(orig=gauss_psf[arm])
    psf.sigma = np.sqrt(gauss_psf[arm].sigma**2 - xsl_sigma(gauss_psf[arm].wave)**2)
    deg_psf[arm] = psf

In [0]:
f, ax = plt.subplots(1, 1, figsize=(3.4, 2.5), dpi=240)

for arm in ARMS:
    ax.plot(gauss_psf[arm].wave, gauss_psf[arm].sigma, lw=0.5)
    ax.plot(gauss_psf[arm].wave, deg_psf[arm].sigma, '--k', lw=0.5)

ax.grid()

ax.set_xlabel('wavelength [A]')
ax.set_ylabel('LSF sigma')

In [0]:
from pfs.ga.pfsspec.sim.obsmod.detectors import PfsDetector
from pfs.ga.pfsspec.sim.obsmod.detectormaps import PfsDetectorMap
from pfs.ga.pfsspec.core.obsmod.resampling import FluxConservingResampler

detector = {}

for arm in ARMS:
    detector[arm] = PfsDetector()
    detector[arm].load_json(DETECTOR_PATH.format(arm))
    detector[arm].map = PfsDetectorMap()
    detector[arm].map.load(DETECTORMAP_PATH.format(arm[0]))

    # print(arm, detector[arm].map.default_fiberid)
    # print(arm, detector[arm].map.get_wave()[0].shape, detector[arm].map.get_wave()[0][[0, -1]], detector[arm].wave)

In [0]:
from pfs.ga.pfsspec.core.util.copy import safe_deep_copy

spec = xshooter.spectra[12]

nspec = type(spec)()
nspec.wave = safe_deep_copy(spec.wave) * 10
#nspec.wave_edges = safe_deep_copy(spec.wave_edges)
nspec.flux = safe_deep_copy(spec.flux)
nspec.flux_err = safe_deep_copy(spec.flux_err)
nspec.flux_dered = safe_deep_copy(spec.flux_dered)

res = FluxConservingResampler()
wave, wave_edges = detector['mr'].get_wave()

nspec.convolve_psf(deg_psf['mr'])
nspec.apply_resampler(res, wave, wave_edges)

In [0]:
spec.flux_err.max(), nspec.flux_err.max()

In [0]:
f, ax = plt.subplots(1, 1, figsize=(3.4, 2.5), dpi=240)

#ax.plot(nspec.wave, nspec.flux, '-r', lw=0.5)
ax.plot(nspec.wave, nspec.flux_err, '-r', lw=0.5)

#ax.plot(spec.wave * 10, spec.flux, '-k', lw=0.5)
ax.plot(spec.wave * 10, spec.flux_err, '-k', lw=0.5)

ax.set_xlim(8450, 8700)
ax.set_ylim(0, 0.2e-13)

In [0]:
spec.Fe_H, spec.T_eff, spec.log_g

In [0]:
model = grid.get_nearest_model(M_H=spec.Fe_H, T_eff=spec.T_eff, log_g=spec.log_g, a_M=0)

In [0]:
f, ax = plt.subplots(1, 1, figsize=(3.4, 2.5), dpi=240)

#mask = (790 <= wave) & (wave <= 796)
#mask = np.full_like(spec.wave, True, dtype=np.bool)

ax.plot(spec.wave * 10, spec.flux, '-k', lw=0.5)
ax.plot(nspec.wave, nspec.flux, '-r', lw=0.5)
ax.plot(model.wave, model.flux * 0.5e-26, '-b', lw=0.5)

# ax.set_xlim(7000, 9000)   # mr
ax.set_xlim(8450, 8700)
ax.set_ylim(0, 1.25e-12)
#ax.set_ylim(0, 1.5 * np.nanquantile(spec.flux[mask], 0.99))

#ax.plot(nspec.wave, nspec.flux_err, '-r', lw=0.5)
#ax.plot(spec.wave * 10, spec.flux_err, '-k', lw=0.5)

ax.set_xlabel('wavelength [A]')
ax.set_ylabel(r'$F_\lambda$ [erg/s/cm2/A]')

ax.set_title('[Fe/H]={}, T_eff={}, log_g={}'.format(spec.Fe_H, spec.T_eff, spec.log_g))

ax.grid()

f.tight_layout()

In [0]:
mask = (detector['mr'].wave[0] <= spec.wave * 10) & (spec.wave * 10 < detector['mr'].wave[1])
snr = spec.flux[mask] / spec.flux_err[mask]
snr_xs = np.quantile(snr, 0.95)
snr_xs

In [0]:
nspec.flux_err

In [0]:
mask = (detector['mr'].wave[0] <= nspec.wave) & (nspec.wave < detector['mr'].wave[1])
snr = nspec.flux[mask] / nspec.flux_err[mask]
snr_pfs = np.quantile(snr, 0.95)
snr_pfs

# Add back noise to match SNR

In [0]:
f, ax = plt.subplots(1, 1, figsize=(3.4, 2.5), dpi=240)

#mask = (790 <= wave) & (wave <= 796)
#mask = np.full_like(spec.wave, True, dtype=np.bool)

#ax.plot(spec.wave * 10, spec.flux, '-k', lw=0.5)
#ax.plot(nspec.wave, nspec.flux, '-r', lw=0.5)
#ax.plot(model.wave, model.flux * 0.5e-26, '-b', lw=0.5)

ax.plot(nspec.wave, nspec.flux, '-k', lw=0.5)
ax.plot(nspec.wave, nspec.flux + 10 * nspec.flux_err * np.random.normal(size=nspec.wave.shape), '-r', lw=0.5)

# ax.set_xlim(7000, 9000)   # mr
ax.set_xlim(8450, 8700)
ax.set_ylim(0, 1.25e-12)
#ax.set_ylim(0, 1.5 * np.nanquantile(spec.flux[mask], 0.99))

#ax.plot(nspec.wave, nspec.flux_err, '-r', lw=0.5)
ax.plot(spec.wave * 10, spec.flux_err * 10, '-k', lw=0.5)

ax.set_xlabel('wavelength [A]')
ax.set_ylabel(r'$F_\lambda$ [erg/s/cm2/A]')

ax.set_title('[Fe/H]={}, T_eff={}, log_g={}'.format(spec.Fe_H, spec.T_eff, spec.log_g))

ax.grid()

f.tight_layout()